In [29]:
library(data.table)

In [30]:
dt <- fread('out.A.tsv')

In [31]:
head(dt)

,V1,V2,V3,V4,V5,V6,V7
1,H5JCKCCXX:2:14:3646328:0/2,A*24:281,205,17,1,1,TDAPKTHMTHHAVSDHE
2,H5JCKCCXX:2:14:3646328:0/2,A*26:79,205,17,17,1,TDAPKTHMTHHAVSDHE
3,H5JCKCCXX:2:14:3646328:0/2,A*24:245,205,17,1,1,TDAPKTHMTHHAVSDHE
4,H5JCKCCXX:2:14:3646328:0/2,A*02:335,205,17,17,1,TDAPKTHMTHHAVSDHE
5,H5JCKCCXX:2:14:3646328:0/2,A*02:124,205,17,17,1,TDAPKTHMTHHAVSDHE
6,H5JCKCCXX:2:14:3646328:0/2,A*32:05,205,17,17,1,TDAPKTHMTHHAVSDHE


In [32]:
setnames(dt, c('q', 't', 'pos', 'len', 'align', 'specific', 'seq'))

In [33]:
nrow(dt)

[1] 123588

In [34]:
tt <- dt[, qp := sub('/\\d$', '', q)]
nr <- dt[, .(pair = length(unique(q))), keyby = qp]
setkey(dt, qp)
dt <- nr[dt]
head(dt)

,qp,pair,q,t,pos,len,align,specific,seq
1,H5JCKCCXX:1:10:2359481:0,1,H5JCKCCXX:1:10:2359481:0/1,A*24:26,206,34,34,1,DPPKTHMTHHPISDHEATLRCWALGFYPAEITLT
2,H5JCKCCXX:1:10:2359481:0,1,H5JCKCCXX:1:10:2359481:0/1,A*24:44,206,34,34,1,DPPKTHMTHHPISDHEATLRCWALGFYPAEITLT
3,H5JCKCCXX:1:10:2359481:0,1,H5JCKCCXX:1:10:2359481:0/1,A*24:135,206,34,34,1,DPPKTHMTHHPISDHEATLRCWALGFYPAEITLT
4,H5JCKCCXX:1:10:2359481:0,1,H5JCKCCXX:1:10:2359481:0/1,A*03:159,206,34,34,1,DPPKTHMTHHPISDHEATLRCWALGFYPAEITLT
5,H5JCKCCXX:1:10:2359481:0,1,H5JCKCCXX:1:10:2359481:0/1,A*23:69,206,34,34,1,DPPKTHMTHHPISDHEATLRCWALGFYPAEITLT
6,H5JCKCCXX:1:10:2359481:0,1,H5JCKCCXX:1:10:2359481:0/1,A*30:93,206,34,34,1,DPPKTHMTHHPISDHEATLRCWALGFYPAEITLT


In [35]:
nr <- dt[, .(pair2 = length(unique(q))), keyby = .(qp, t)]
setkey(dt, qp, t)
dt <- nr[dt]
head(dt)

,qp,t,pair2,pair,q,pos,len,align,specific,seq
1,H5JCKCCXX:1:10:2359481:0,A*01:01,1,1,H5JCKCCXX:1:10:2359481:0/1,206,34,34,1,DPPKTHMTHHPISDHEATLRCWALGFYPAEITLT
2,H5JCKCCXX:1:10:2359481:0,A*01:02,1,1,H5JCKCCXX:1:10:2359481:0/1,206,34,34,1,DPPKTHMTHHPISDHEATLRCWALGFYPAEITLT
3,H5JCKCCXX:1:10:2359481:0,A*01:03,1,1,H5JCKCCXX:1:10:2359481:0/1,206,34,34,1,DPPKTHMTHHPISDHEATLRCWALGFYPAEITLT
4,H5JCKCCXX:1:10:2359481:0,A*01:09,1,1,H5JCKCCXX:1:10:2359481:0/1,206,34,34,1,DPPKTHMTHHPISDHEATLRCWALGFYPAEITLT
5,H5JCKCCXX:1:10:2359481:0,A*01:104,1,1,H5JCKCCXX:1:10:2359481:0/1,206,34,34,1,DPPKTHMTHHPISDHEATLRCWALGFYPAEITLT
6,H5JCKCCXX:1:10:2359481:0,A*01:107,1,1,H5JCKCCXX:1:10:2359481:0/1,206,34,34,1,DPPKTHMTHHPISDHEATLRCWALGFYPAEITLT


In [36]:
dt <- dt[pair == pair2]

In [37]:
library(IRanges)

In [38]:
dt <- dt[grepl('\\d$', t) & align / len > 0.8]
setkey(dt, t)
cov <- dt[, .(
    n0 = .N, 
    n1 = sum(specific), 
    cov0 = sum(width(reduce(IRanges(pos, width = len)))),
    cov1 = sum(width(reduce(IRanges(pos[specific == 1], width = len[specific == 1]))))
), keyby = t]

In [39]:
head(cov[order(-cov1)])

,t,n0,n1,cov0,cov1
1,A*02:01,99,88,297,297
2,A*02:249,84,75,296,296
3,A*02:523,85,74,296,296
4,A*02:95,79,68,296,296
5,A*02:134,98,87,293,293
6,A*02:24,87,76,292,292


In [40]:
head(cov[order(-n1)])

,t,n0,n1,cov0,cov1
1,A*03:01,176,108,298,253
2,A*03:134,166,108,287,253
3,A*03:26,174,108,295,253
4,A*03:182,132,106,266,251
5,A*03:37,128,106,263,251
6,A*03:89,175,105,298,253


In [41]:
pure <- dt[specific == 1]
nonpure <- dt[specific == 0]

In [42]:
nrow(pure)

[1] 67598

In [43]:
nrow(nonpure)

[1] 24624

In [44]:
#good <- pure[align / len > 0.8]
good <- dt[align / len > 0.8]
nrow(good)

[1] 92222

In [54]:
tt <- good[, v := (specific + 1)/2]

In [55]:
mat <- dcast(good, q ~ t, value.var = 'v', fill = 0)

In [56]:
tt <- mat[, q := NULL]

In [57]:
tail(sort(colSums(mat)))

A*03:196  A*03:20 A*03:134  A*03:89  A*03:26  A*03:01 
   131.0    134.5    137.0    140.0    141.0    142.0

In [58]:
temp <- sort(colSums(mat))

In [62]:
tail(temp[grepl('A\\*02', names(temp))])

A*02:329 A*02:445 A*02:446 A*02:134  A*02:01 A*02:294 
    90.5     90.5     91.0     92.5     93.5    100.5

In [59]:
mat[, sum(pmax(`A*02:01`, `A*03:01`))]

[1] 212.5

In [60]:
mat[, sum(pmax(`A*02:01`, `A*03:26`))]

[1] 211.5

In [61]:
mat[, sum(pmax(`A*02:01`, `A*03:89`))]

[1] 193.5

In [64]:
mat[, sum(pmax(`A*02:294`, `A*03:89`))]

[1] 188

In [63]:
mat[, sum(pmax(`A*02:294`, `A*03:01`))]

[1] 207

In [65]:
mat[, sum(pmax(`A*02:294`, `A*03:26`))]

[1] 208

In [66]:
mat[, sum(pmax(`A*02:134`, `A*03:26`))]

[1] 210.5

In [67]:
mat[, sum(pmax(`A*02:134`, `A*03:01`))]

[1] 211.5

In [55]:
nrow(mat)

[1] 151

In [100]:
head(cov)

,t,n0,n1,cov0,cov1
1,B*07:02,43,16,92,92
2,B*07:03,43,16,92,92
3,B*07:04,43,16,92,92
4,B*07:05,43,16,92,92
5,B*07:06,43,16,92,92
6,B*07:07,43,16,92,92


In [24]:
mat[, sum(pmax(`A*02:01`, `A*03:01`))]

[1] 176

In [25]:
mat[, sum(pmax(`A*02:01`, `A*03:26`))]

[1] 176

In [26]:
mat[, sum(pmax(`A*02:01`, `A*03:14`))]

[1] 131

In [27]:
mat[, sum(pmax(`A*02:01`, `A*03:134`))]

[1] 176

In [28]:
mat[, sum(pmax(`A*02:01`, `A*03:37`))]

[1] 174